In [7]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [9]:

data_dir = '/Users/thankaselv.kumaresan/Downloads/Cat_Dog_data'

random_transforms = [transforms.RandomRotation(25),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.ColorJitter(0.5, 0.5, 0.5, 0.5)]
train_transforms = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.RandomApply(random_transforms, p=0.6),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])

train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=32)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)

In [11]:
model = models.densenet121(pretrained=True)

In [17]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.2)),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
criterion = nn.NLLLoss()
# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

model.to(device)
epochs = 1
for e in range(epochs):
    running_loss = 0
    for inputs, labels in trainloader:
        # Move input and label tensors to the device
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        # training
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else :
        with torch.no_grad():
            model.eval()
            test_loss = 0
            for images, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                test_output = model(images)
                test_loss += criterion(test_output, labels)
                top_p, top_class = test_log_prob.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy = torch.mean(equals.type(torch.FloatTensor))
                print(f'Accuracy: {accuracy.item()*100}%')
        model.train()